# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
ds = pd.read_csv('/datasets/data.csv')
ds.info()
display(ds.head())
ds.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


children                8
days_employed       19351
dob_years              58
education              15
education_id            5
family_status           5
family_status_id        5
gender                  3
income_type             8
debt                    2
total_income        19351
purpose                38
dtype: int64

**Вывод**

Полученные данные не пригодны для качественного анализа. В столбцах **"days_employed"** и **"total_income"** отсутствуют данные, вещественные значения в этих же столбцах мешают воспритию данных. В столбце **"education"** не соблюдается единый регистр заполнения, что вполне могло привести к появлению дубликатов, количество уникальных значений этого столбца должно совпадать со столбцом **"education_id"**. Дубликаты могли возникнуть и на основе последнего столбца **"purpose"**, где значения скорее всего заполнялись в свободной форме. В столбце **"gender"** присутствует третье лишнее обозначение пола, нужно его найти и исправить.  Стоит проверить на наличие ошибок столбцы **"children"** и **"dob_years"**.  Следует разобраться и со столбцом **"income_type"** - непонятны принципы категоризации данных в этом столбце. Поэтому перед началом анализа полученные данные следует обработать.

## Шаг 2. Предобработка данных

### Обработка пропусков

Явные пропуски присутствуют в столбцах **"days_employed"** и **"total_income"**, обработаем их.

In [2]:
display(ds[ds['days_employed'].isna()].head())
round(len(ds[ds['days_employed'].isna()])/len(ds['days_employed'])*100, 1) # количество пропусков в процентном отношении

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


10.1

Пропуски в столбцах **"days_employed"** и **"total_income"** приходятся на одни и те же строки (данные по клиенту). Это может быть связано с невозможностью или нежеланием заемщика передавать банку информацию о трудовом стаже и доходе. Количество пропусков составляет около 10%, если строки с пропущенными значениями удалить, то это может привести к искажению общего результата анализа. Однозначно связать наличие пропусков с данными одного из столбцов нельзя, пропущенные значения встречаются у разных заемщиков, независимо от семейного положения, типа занятости, пола, количества детей или цели кредита. Но имеет смысл связать пропущенные значения с типом занятости заемщика - найти медиану значений **"days_employed"** и **"total_income"** по каждой категории из столбца **"income_type"** и заменить пропущенные данные полученными значениями.

In [3]:
ds_grouped_days = ds.groupby('income_type')['days_employed'].median() # медианы значений общего трудового стажа по категориям
def replacement(row):                                        # функция для замены пропущенных значений в столбце "income_type"
    if pd.isna(row['days_employed']):
        return ds_grouped_days.loc[row['income_type']]
    return row['days_employed']
ds['days_employed'] = ds.apply(replacement, axis=1)

ds_grouped_income = ds.groupby('income_type')['total_income'].median() # медианы значений ежемесячного дохода по категориям
def replacement(row):                                        # функция для замены пропущенных значений в столбце "total_income"
    if pd.isna(row['total_income']):
        return ds_grouped_income.loc[row['income_type']]
    return row['total_income']
ds['total_income'] = ds.apply(replacement, axis=1)           
ds[ds.isna()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Далее проверим столбцы **"children"**, **"dob_years"** и **"gender"** на наличие ошибок и исправим их.

In [4]:
display(ds.groupby('children')['children'].count()) # количество уникальных значений в столбце
ds.loc[ds['children'] < 0, 'children'] = abs(ds['children']) # исправляем отрицательные значения
ds.loc[ds['children'] > 5, 'children'] = ds['children']/10 # исправляем значения с лишним знакомом 0
ds.groupby('children')['children'].count() # проверяем количество уникальных значений после исправлений

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

children
0.0    14149
1.0     4865
2.0     2131
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

В столбце **"children"** имелись две ошибки: -1 и 20, которые возникли скорее всего при внесении данных. Отрицательного значения количества детей быть не может и 76 заемщиков с 20-ю детьми скорее всего тоже. 

Далее проверим столбец **"dob_years"**: 

In [5]:
display(ds.groupby('dob_years')['dob_years'].count()) # количество уникальных значений в столбце
ds.loc[ds['dob_years'] == 0, 'dob_years'] = ds['dob_years'].median() # исправляем нулевые значения на медианные
ds.groupby('dob_years')['dob_years'].count() # проверяем уникальные значения после исправлений

dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

dob_years
19.0     14
20.0     51
21.0    111
22.0    183
23.0    254
24.0    264
25.0    357
26.0    408
27.0    493
28.0    503
29.0    545
30.0    540
31.0    560
32.0    510
33.0    581
34.0    603
35.0    617
36.0    555
37.0    537
38.0    598
39.0    573
40.0    609
41.0    607
42.0    698
43.0    513
44.0    547
45.0    497
46.0    475
47.0    480
48.0    538
49.0    508
50.0    514
51.0    448
52.0    484
53.0    459
54.0    479
55.0    443
56.0    487
57.0    460
58.0    461
59.0    444
60.0    377
61.0    355
62.0    352
63.0    269
64.0    265
65.0    194
66.0    183
67.0    167
68.0     99
69.0     85
70.0     65
71.0     58
72.0     33
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

В столбце **"dob_years"** имелась 101 строка с данными о заемщике с возрастом 0 лет. Доподлинно установить возраст у нас возможности нет, поэтому стоит изменить значние возраста в этих строках на медианное по всему столбцу. После изменений количество заемщиков в возрасте 42 года (медиана) увеличилось на 101 человека. Данные исказились, но поскольку изменения пришлись самую активную возрастную группу заемщиков, можно считать, что на конечный результат такое измение не повлияет.

В столбце **"gender"** явно имеется ошибка, нужно и её исправить:

In [6]:
display(ds['gender'].value_counts()) 
ds.loc[ds['gender'] == 'XNA', 'gender'] = 'F' 
ds['gender'].value_counts() 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

F    14237
M     7288
Name: gender, dtype: int64

Поскольку заемщиков женского пола вдвое больше и неопределенное обозначение "XNA" присутствует только в единичном случае, можно изменить значение "XNA" на "F", на общую картину такое изменение не повлияет. 

**Вывод**

Теперь пустых значений в таблице нет, явных ошибочных данных тоже нет. В столбцах **"days_employed"** и **"total_income"** тип указанных данных - вещественный, с шестью знаками после запятой. Для ответа на поставленные гипотезы точность данных из столбцов **"days_employed"** и **"total_income"** особой роли не играет, поэтому имееет смысл привести тип данных в этих столбцах к целому. В столбце **"children"** после изменений тип данных сменился на вещественный, его тоже стоит вернуть к целому.

### Замена типа данных

In [7]:
ds['days_employed'] = ds['days_employed'].astype('int')
ds['total_income'] = ds['total_income'].astype('int')
ds['children'] = ds['children'].astype('int')
ds.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Значения в обработанных столбцах достаточно велики, чтобы данными после запятой можно было пренебречь. Осталось разобраться со столбцом **"days_employed"**(общий трудовой стаж в днях) - каким образом подсчитывались и вносились указанные данные? Отрицательных значений трудового стажа быть не может, как и нереально огромных значений, встречающихся в таблице (например, 5-я строка ds: 340266 дней - это больше 932 лет!!!)

In [8]:
display(ds.groupby('income_type')['days_employed'].max())
display(ds.groupby('income_type')['days_employed'].min())
ds.groupby('income_type')['days_employed'].count()

income_type
безработный        395302
в декрете           -3296
госслужащий           -39
компаньон             -30
пенсионер          401755
предприниматель      -520
сотрудник             -24
студент              -578
Name: days_employed, dtype: int64

income_type
безработный        337524
в декрете           -3296
госслужащий        -15193
компаньон          -17615
пенсионер          328728
предприниматель      -520
сотрудник          -18388
студент              -578
Name: days_employed, dtype: int64

income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: days_employed, dtype: int64

Что все-таки означают данные из столбца **"days_employed"** точно определить на основе имеющихся нельзя. У большинства групп заемщиков, кроме "безработных" и "пенсионеров" значения в столбце о трудовом стаже отрицательные, а у указанных слишком огромные положительные. Можно предположить, что при формировании датафрейма произошла ошибка в наименовании столбца - он явно содержит данные не о количестве дней трудового стажа, а некоторую другую информацию, возможно внутрибанковскую шкалу оценки заемщиков или что-то подобое. Уточнить эту информацию сейчас не представляется возможным, данные этого столбца для решения нашей задачи не играют особого значения, поэтому можно оставить столбец без дальнейших изменений.

### Обработка дубликатов

In [9]:
display(ds.duplicated().sum()) # проверяем на наличие дубликатов 
ds['education'] = ds['education'].str.lower() # приводим значения к однообразому нижнему регистру
display(ds.duplicated().sum()) # проверяем на наличие дубликатов 
ds = ds.drop_duplicates() # удаляем явные дубликаты
ds = ds.dropna().reset_index(drop=True) # удаляем пропуски, обновляем индексы
display(ds.duplicated().sum()) # проверяем на наличие дубликатов
ds.info()

55

72

0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  int64  
 2   dob_years         21453 non-null  float64
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  int64  
 11  purpose           21453 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Датасет включал в себя 55 явных дубликатов, возникших в результате ошибки при внесении данных. После приведения значений в столбце **"education"** к однообразному регистру, появилась возможность определить еще 17 явных дубликатов. После удаления явных дубликатов таблица сократилась на 72 строку. 

На наличие неявных дубликатов следует проверить датасет по столбцам **"family_status"** и **"purpose"**.

In [10]:
display(ds['family_status'].unique())
ds['family_status'] = ds['family_status'].str.lower()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Со столбцом **"family_status"** все в порядке, дубликатов возникнуть здесь не может. Для однообразности записи данных можно привести значения столбца к нижнему регистру.

In [11]:
ds['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

А вот в столбец **"purpose"** данные по всей видимости вносились в свободной форме. Все указанные значения можно сгруппировать в 4 категории: недвижимость (здесь же и жилье), автомобиль, образование и свадьба.

### Лемматизация

In [12]:
from pymystem3 import Mystem
m = Mystem() 
ds['purpose_lemmas'] = ds['purpose'].apply(lambda text: m.lemmatize(text)) # создадим столбец с леммами из столбца purpose
ds.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


**Вывод**

Полученный столбец с леммами позволяет разделить цели кредита на 4 группы.  

### Категоризация данных

In [13]:
def purpose_group(lemma): # функция поиска лемм для группировки по целям кредита
    if 'жилье' in lemma:
        return 'недвижимость'
    if 'недвижимость' in lemma:
        return 'недвижимость'
    if 'автомобиль' in lemma:
        return 'автомобиль'
    if 'образование' in lemma:
        return 'образование'
    if 'свадьба' in lemma:
        return 'свадьба'
    return 'категория не определена'
ds['purpose_group'] = ds['purpose_lemmas'].apply(purpose_group)
ds.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_group
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


После категоризации данных по столбцу **"purpose_group"**, можем удалить из датасета столбцы **"purpose"** и **"purpose_lemmas"** и еще раз проверить на наличие дубликатов.

In [14]:
ds_new = ds.drop(['purpose', 'purpose_lemmas'], axis=1) # создадим новую таблицу без ненужных столбцов
display(ds_new.duplicated().sum()) # находим дубликаты, удаляем их с обновлением индексов
ds_new = ds_new.drop_duplicates() 
ds_new = ds_new.dropna().reset_index(drop=True)
display(ds_new.head())

336

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_group
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


Для ответа на вопросы необходимо категоризиировать данные по столбцам **"children"** и **"total_income"**. По столбцу **"children"** сгруппируем заемщиков по количеству детей таким образом: 0 - "нет детей", 1-2 - "1-2 ребенка", 3 и более - "многодетные".


In [15]:
def children_group(child): 
    if child == 0:
        return 'нет детей'
    if 3 > child > 0 :
        return '1-2 ребенка'
    if child >= 3:
        return 'многодетные'
ds_new['children_group'] = ds_new['children'].apply(children_group)
ds_new.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_group,children_group
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1-2 ребенка
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1-2 ребенка
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,нет детей
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,многодетные
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей


Далее нужно категоризировать заемщиков по уровню дохода. Разница между минимальным и максимальным значениями огромная, для более наглядного рассмотрения, разделим уровни дохода на 5 категорий:

"низкий" < 50000

"ниже среднего" < 100000

"средний" < 250000

"выше среднего" < 500000

"высокий" >= 500000

In [16]:
def income_group(income): # создаем функцию для категоризации по уровню дохода
    if income < 70000:
        return 'низкий'
    if income < 120000:
        return 'ниже среднего'
    if income < 250000:
        return 'средний'
    if income < 500000:
        return 'выше среднего'
    return 'высокий'
ds_new['income_category'] = ds_new['total_income'].apply(income_group) # новый столбец с категориями заемщиков по уровню дохода
ds_new.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_group,children_group,income_category
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1-2 ребенка,выше среднего
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1-2 ребенка,ниже среднего
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,нет детей,средний
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,многодетные,выше среднего
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей,средний


**Вывод**

Категоризация данных по цели кредита позволила обнаружить еще 336 дубликатов в таблице. Перед ответами на поставленные вопросы перезапишем новый датасет, изменив расположение столбцов **"children_group"** и **"income_category"**, столбец **"debt"** переместив в конец и удалив ненужные для окончательного анализа столбцы **"days_employed"**, **"dob_years"**, **"education"**, **"education_id"**, **"gender"**, **"income_type"**:

In [17]:
ds_new = ds_new[['children', 'children_group', 'family_status', 'family_status_id', 'total_income', 'income_category', 'purpose_group', 'debt']]
ds_new.head()

,children,children_group,family_status,family_status_id,total_income,income_category,purpose_group,debt
0,1,1-2 ребенка,женат / замужем,0,253875,выше среднего,недвижимость,0
1,1,1-2 ребенка,женат / замужем,0,112080,ниже среднего,автомобиль,0
2,0,нет детей,женат / замужем,0,145885,средний,недвижимость,0
3,3,многодетные,женат / замужем,0,267628,выше среднего,образование,0
4,0,нет детей,гражданский брак,1,158616,средний,свадьба,0


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
ds_new.groupby('children_group')['debt'].mean()

children_group
1-2 ребенка    0.093592
многодетные    0.082011
нет детей      0.076739
Name: debt, dtype: float64

**Вывод**

Сгруппировав заемщиков по наличию детей и вычислив долю просроченных платежей среди них выясняется, что просрочки возникают у всех заемщиков независимо от наличия и количества детей. Разница в своевременности оплаты между рассматриваемыми группами заемщиков совсем небольшая, поэтому делаем вывод, что наличие детей не влияет на оплату кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
display(pd.pivot_table(ds_new, index=['family_status'], values=['family_status_id', 'debt'], aggfunc={'family_status_id' : len, 'debt': [sum, 'mean'] }))   


debt        family_status_id
                           mean    sum              len
family_status                                          
в разводе              0.071249   85.0             1193
вдовец / вдова         0.066808   63.0              943
гражданский брак       0.094106  388.0             4123
женат / замужем        0.076942  929.0            12074
не женат / не замужем  0.098420  274.0             2784

**Вывод**

Отталкиваясь от семейного положения заемщиков также невозможно утверждать, что семейное положение влияет на возврат кредита в срок. Просроченные платежи встречаются у всех групп заемщиков, чаще всего у "не женатых/не замужних" и находящихся в "гражданском браке". Но, как и в прошлом пункте, разница незначительна, поэтому делаем вывод - зависимости между семейным положением и возврата кредита в срок нет.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [20]:
pd.pivot_table(ds_new, index=['income_category'], values=['total_income','debt'], aggfunc={'total_income' : len, 'debt': [sum, 'mean'] }) 


debt        total_income
                     mean    sum          len
income_category                              
высокий          0.063063   14.0          222
выше среднего    0.069471  180.0         2591
ниже среднего    0.085689  485.0         5660
низкий           0.068521  101.0         1474
средний          0.085855  959.0        11170

**Вывод**

Уровень дохода скорее всего влияет на доступную сумму для кредита, а на погашение кредита в срок - нет.

- Как разные цели кредита влияют на его возврат в срок?

In [21]:
ds_new['purpose_group_id'] = ds_new['purpose_group']
display(pd.pivot_table(ds_new, index=['purpose_group'], values=['purpose_group_id','debt'], aggfunc={'purpose_group_id' : len, 'debt': [sum, 'mean'] }))  
ds_new = ds_new.drop(['purpose_group_id'], axis=1)

debt        purpose_group_id
                   mean    sum              len
purpose_group                                  
автомобиль     0.094101  402.0             4272
недвижимость   0.073846  781.0            10576
образование    0.093340  370.0             3964
свадьба        0.080694  186.0             2305

**Вывод**

И цель кредита не оказвает должного влияния на возврат кредита в срок. 

## Шаг 4. Общий вывод

Полученные исходные данные не были пригодны для качественного анализа и ответа на поставленные вопросы. В ходе предобработки данных был выявлен и устранен ряд недочетов:
- пропуски в количественных переменных были заменены на медианные, сгруппированнные по типу
- в данных о наличии и количестве детей были обнаружены и устранены значения, явно не соответствующие действительности (артефакты)
- такие же явные ошибки в данных были обнаружены и устранены в столбцах о возрасте и поле заемщика
- для выявления и удаления дубликатов понадобилось привести значения к единому регистру, а также сгруппировать заемщиков по цели кредита, предварительно лемматизировав исходные значения
- для ответа на поставленные вопросы провели категоризацию данных по наличию детей и их количеству, а также по уровню дохода заемщиков, разделив их на 5 групп

В результате проведенного анализа можно сделать вывод: наличие детей, семейное положение, уровень дохода и цель кредита не оказывают должного влияния на возврат кредита в срок. Между категориями каждого из перечисленных критериев есть разница, но она не превышает 3%, т.е. просроченные платежи по кредитам встречаются в каждой группе заемщиков в примерно равном отношении.